# Email me if something isnt working Ill respond right away: prsood@sfu.ca

First download the model from SFU vault with link: 
https://vault.sfu.ca/index.php/s/w3tSbMVnBJ5WNkh


Next download the testing data from SFU vault with link:
https://vault.sfu.ca/index.php/s/BN4Od2Rin1aXEFz


From here, open up the zip file and then type in the relative path to the image files that you have created to wherever you need to run them.

Finally download the pickle file associated with them:
https://vault.sfu.ca/index.php/s/7yKBdl0tJsXDO4r



In [ ]:
import sys
sys.path.insert(0,"/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/") 
__package__ = 'EvaluationScripts'

In [ ]:
from SingleModels.models.text import BertClassifier
from SingleModels.models.image import ResnetClassification
from DoubleModels.models.visualBert import VBertClassifier
from utils.data_loaders import BertDataset, VBertDataset , ImageDataset
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from SingleModels.train_model.text_training import evaluate_text
from SingleModels.train_model.image_training import evaluate_img
from DoubleModels.train_model.visual_bert_train import evaluate_vbert
from utils.global_functions import Metrics
import torch



In [ ]:
args = {}
args['output_dim'] = 2
model = VBertClassifier(args)
model.load_state_dict(torch.load(PATH)) # GPU
#  model.load_state_dict(torch.load(PATH,map_location=torch.device('cpu'))) # CPU

In [ ]:
def prepare_dataloader(df , batch_size, pin_memory=True, num_workers=4):
    """
    we load in our dataset, and we just make a random distributed sampler to evenly partition our 
    dataset on each GPU
    say we have 32 data points, if batch size = 8 then it will make 4 dataloaders of size 8 each 
    """
    max_len = 70 # just max number of tokens from LSTM    keep this line in here somewhere

    dataset = VBertDataset(df , max_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, pin_memory=pin_memory, 
            num_workers=num_workers, drop_last=False, shuffle=True)
    return dataloader

Read in pickle file from wherever you stored it

In [ ]:
df = pd.read_pickle(f"{PATH}")

Now change the img_path to be where you stored the images

In [ ]:
df['img_path'].apply(lambda x: "HARD_PATH_TO_IMG_FOLDER" + x[15:])
                                            # DONT TOUCH THE SPLICE

Create our partition to get the testing data then create the dataloader for test

In [ ]:
param_dict = {'epoch': 5, 'patience': 10, 'lr': 0.1, 'clip': 1, 'batch_size': 16, 'weight_decay': 1e-06, 'model': 'VisualBert', 'T_max': 5, 'seed': 64, 'weights': torch.Tensor([0.3667, 0.6333]), 'label2id': {'Negative': 0, 'Positive': 1}, 'id2label': {0: 'Negative', 1: 'Positive'}}
model_param = {'input_dim': 2, 'output_dim': 2, 'lstm_layers': 1, 'hidden_layers': [300]}

In [ ]:
df_train, df_test, _, __ = train_test_split(df, df["label"], test_size = 0.25, random_state = param_dict['seed'] , stratify=df["label"])
dataloader_test = prepare_dataloader(df_test , batch_size=param_dict["batch_size"])

Evaluate the model

In [ ]:
id2label = {0: 'Negative', 1: 'Positive'}
# Metric = Metrics(num_classes = 2, id2label = id2label) #CPU
Metric = Metrics(num_classes = 2, id2label = id2label , rank = "cuda") # GPUlocation = <PATH YOU WANT TO SEE TEST RESULTS>
evaluate_vbert(model, df_test, Metric , location )

In [ ]:
with open(location) as f:
    lines = f.readlines()

At the very end of the file you can find a confusino matrix